In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

Using TensorFlow backend.


In [2]:
def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train2", image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)
    train_generator.reset()
    train = model.predict_generator(train_generator, verbose=1)
    test_generator.reset()
    test = model.predict_generator(test_generator, verbose=1)

    with h5py.File("gap_InceptionV3.h5") as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

In [6]:
write_gap(ResNet50, (224, 224))

C:\ProgramData\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 4028 images belonging to 2 classes.
Found 0 images belonging to 1 classes.
252/252 [==============================] - 746s 3s/step


In [8]:
write_gap(Xception, (299, 299), xception.preprocess_input)

W0418 15:29:07.713725 12448 deprecation.py:237] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4421: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Found 4028 images belonging to 2 classes.
Found 0 images belonging to 1 classes.
252/252 [==============================] - 1436s 6s/step


In [3]:
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)

W0418 15:55:15.406211 14372 deprecation.py:237] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4421: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0418 15:55:15.454170 14372 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\training\moving_averages.py:211: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Found 4028 images belonging to 2 classes.
Found 0 images belonging to 1 classes.
252/252 [==============================] - 583s 2s/step


In [ ]:
write_gap(VGG16, (224, 224))

In [ ]:
write_gap(VGG19, (224, 224))